<div style="display: flex; align-items: center; gap: 40px;">

<img src="https://play-lh.googleusercontent.com/_O9p4Z4yucA2NLmZBu9mTJCuBwXeT9NcbtrDN6I8gKlkIPRySV0adOmbyipjSj9Gew" width="130">



<div>
  <h2>SUTRA by TWO Platforms </h2>
  <p>SUTRA is a family of large multi-lingual language (LMLMs) models pioneered by Two Platforms. SUTRA’s dual-transformer approach extends the power of both MoE and Dense AI language model architectures, delivering cost-efficient multilingual capabilities for over 50+ languages. It powers scalable AI applications for conversation, search, and advanced reasoning, ensuring high-performance across diverse languages, domains and applications.</p>


<div>
  <h2>Hypothetical Document Embeddings (HyDE) RAG Using SUTRA</h2>
  <p>HyDE operates by creating hypothetical document embeddings that represent ideal documents relevant to a given query. This method contrasts with conventional RAG systems, which typically rely on the similarity between a user's query and existing document embeddings. By generating these hypothetical embeddings, HyDE effectively guides the retrieval process towards documents that are more likely to contain pertinent information.</p>



[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1ADc0aDPlQ2f4gRFBV6NYYBGhFGH9HzqA?usp=sharing)

## Get Your API Keys

Before you begin, make sure you have:

1. A SUTRA API key (Get yours at [TWO AI's SUTRA API page](https://www.two.ai/sutra/api))
2. Basic familiarity with Python and Jupyter notebooks

This notebook is designed to run in Google Colab, so no local Python installation is required.

###🔧 1. Install Required Libraries

In [ ]:
!pip install -qU langchain_openai langchain_community chromadb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 3.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.3/19.3 MB 109.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 25.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.5/438.5 kB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 111.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 6.0 MB/s eta 0:

###🔑 2. Set Environment Variables (API Keys)

In [ ]:
import os
from google.colab import userdata

# Set the API key from Colab secrets
os.environ["SUTRA_API_KEY"] = userdata.get("SUTRA_API_KEY2")
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

###Initialize Openai embeddings

In [ ]:
# load embedding model
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings()

###📄 Load and Chunk Documents

In [ ]:
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = CSVLoader("./context.csv")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
documents = text_splitter.split_documents(documents)

###🔍 Vector Store: ChromaDB

In [ ]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents, embeddings)

###🔁 Retriever from Vectorstore

In [ ]:
retriever = vectorstore.as_retriever()

###Setup Sutra LLM (via ChatOpenAI interface)

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Sutra LLM setup
llm = ChatOpenAI(
    api_key=os.getenv("SUTRA_API_KEY"),
    base_url="https://api.two.ai/v2",
    model="sutra-v2"
)

# Hypothetical Answer Prompt
prompt_hyde = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that answers questions.\nQuestion: {input}\nAnswer:"),
    ("human", "{input}")
])

qa_no_context = prompt_hyde | llm | StrOutputParser()

###Hypothetical Answer Chain (HyDE style hallucination)

In [ ]:
# hallucinate an answer to generate better query
question = "how does interlibrary loan work"
hypothetical_answer = qa_no_context.invoke({"input": question})
print("🧠 Hypothetical Answer:\n", hypothetical_answer)

###Retrieve Relevant Docs Based on HyDE Answer

In [ ]:
# retrieve documents based on hypothetical answer
retrieval_chain = qa_no_context | retriever
retrieved_docs = retrieval_chain.invoke({"input": question})

# Combine content into string for prompt
context_text = "\n".join([doc.page_content for doc in retrieved_docs])

###Final RAG Answer Using Context

In [ ]:
# RAG prompt with context
template_context = """
You are a helpful assistant that answers questions based on the provided context.
Use the provided context to answer the question.
Question: {input}
Context: {context}
"""

prompt_context = ChatPromptTemplate.from_template(template_context)
final_rag_chain = prompt_context | llm | StrOutputParser()

# Final context-aware response
final_answer = final_rag_chain.invoke({"context": context_text, "input": question})
print("✅ Final Answer:\n", final_answer)

###📊 Prepare Data for Evaluation

In [ ]:
from datasets import Dataset
import pandas as pd

# Inference loop (can extend with more questions)
questions = ["how does interlibrary loan work"]
responses, contexts = [], []

for q in questions:
    docs = retriever.invoke(q)
    ctx_text = "\n".join([doc.page_content for doc in docs])
    result = final_rag_chain.invoke({"context": ctx_text, "input": q})

    responses.append(result)
    contexts.append([doc.page_content for doc in docs])

# Create HuggingFace Dataset
data = {"query": questions, "response": responses, "context": contexts}
dataset = Dataset.from_dict(data)

# Convert to DataFrame
df = pd.DataFrame(dataset)
df